Flux with MNIST and GPU
* Pre-treatment with PCA

Libraries

In [ ]:
using CUDA

using MLDatasets           # mnist
using PreprocessingImages; pim = PreprocessingImages

using MLJ                  # make_blobs, rmse, confmat, f1score, coerce
using MLDataUtils          # label, nlabel, labelfreq, stratifiedobs
using Flux                 # the julia ml library

using Random
using Plots; gr()

In [ ]:
include( expanduser("~/projects/pesquisa/libs/libml.jl") )
include( expanduser("~/projects/pesquisa/libs/misc.jl") )

MNIST

In [ ]:
# load mnist
trX,   trY   = MNIST(:train)[:]
testX, testY = MNIST(:test)[:];

In [ ]:
# split train, validation, test sets
Random.seed!(1)
(trainX, trainY), (validationX, validationY) = stratifiedobs((trX, trY), p = 0.7)
size(trainX), size(validationX), size(testX)

Preprocessing functions

In [ ]:
# MLJ works with dataframes
function preprocess1(X, y)
    Xs = permutedims(X, (2, 1, 3))   # needed to adjust visualization of digits
    Xs = pim.batchImage2DF(Xs)
    
    ys = Int32.(y)

    return (Xs, ys)
end

In [ ]:
# reduce predictors
PCA = @load PCA pkg=MultivariateStats verbosity=0
reducer = PCA(pratio = 0.9)

# standardize predictors
std = Standardizer()

# machine
pipe = @pipeline reducer std

In [ ]:
# Flux works with matrices and vectors
function preprocess2(X, y)
    Xs = df2FluxMatrix(X)
    ys = Flux.onehotbatch( Float32.(y), 0:9 )
    return (Xs, ys)
end

Data preprocessing

In [ ]:
X_tr , y_tr  = preprocess1(trainX, trainY)
X_val, y_val = preprocess1(validationX, validationY)

typeof(X_tr), size(X_tr), size(y_tr)

In [ ]:
mach = MLJ.machine(pipe, X_tr) |> fit!
X_tr  = MLJ.transform(mach, X_tr)  .|> Float32   # transform(unsupervised) vs predict(supervised)
X_val = MLJ.transform(mach, X_val) .|> Float32   # transform(unsupervised) vs predict(supervised)

typeof(X_tr), size(X_tr), size(y_tr)

In [ ]:
X_tr , y_tr  = preprocess2(X_tr, y_tr)
X_val, y_val = preprocess2(X_val, y_val)

d, N         = size(X_tr)
typeof(X_tr), size(X_tr), size(y_tr)

In [ ]:
# data on gpu
X_tr_g  = X_tr |> gpu
y_tr_g  = y_tr |> gpu
X_val_g = X_val |> gpu;

Model

In [ ]:
# model architecture
nInputs  = d
nOutputs = 10

# Flux.Dense(nInputs, 256, tanh)
model = Chain( Dense(nInputs => 16, relu),
               Dense(16      => nOutputs, relu),
               softmax)      |> gpu                # weights on gpu

In [ ]:
# training parameters
η = 0.2           # learning rate (\eta<tab>)
lossFunction(X, y) = Flux.mse( model(X), y )
modelParameters    = Flux.params(model)
dataset            = Flux.DataLoader((X_tr_g, y_tr_g), batchsize=32) # batchsize=1 => SGD, batch size > 1 => batch gradient descent
modelOptimizer     = Flux.Descent(η);
# callBack           = Flux.throttle(() -> println("."), 10);        # print every 10s

Training

In [ ]:
numberOfEpochs = 5000

In [ ]:
function predictOutcome(X)
    ŷ = Flux.onecold( model(X), [0:9;] )
end

In [ ]:
# # https://fluxml.ai/Flux.jl/stable/training/training/

# # preferred for multiple epochs
# epochs = 1:numberOfEpochs
# for epoch in epochs
#     Flux.train!(lossFunction, modelParameters, data, modelOptimizer; cb=callBack)
# end

In [ ]:
# https://fluxml.ai/Flux.jl/stable/training/training/

epochs            = 1 : numberOfEpochs
loss_tr           = Vector{Float64}()
acc_val           = Vector{Float64}()
maxLoss           = 1e-4   # hint: start at 1e-2, then go lower for more epochs
nearDeltaZeroLoss = maxLoss / 25
nearDeltaZeroAcc  = 1e-5

for epoch in epochs
    # train
    loss = trainModel!( lossFunction, modelParameters, dataset, modelOptimizer )   # libml
    push!(loss_tr, loss)
    # print(".")   # "epoch completed" indicator

    # predict
    ŷ  = predictOutcome(X_val_g)
    f1 = MLJ.multiclass_f1score(ŷ, coerce(validationY, OrderedFactor))
    push!(acc_val, f1)

    # exit criteria
    if stopTrainingLossCriteria(loss_tr, maxLoss, nearDeltaZeroLoss)   break   end
    # if stopTrainingAccuracyCriteria(acc_val, nearDeltaZeroAcc)         break   end
end

In [ ]:
# plot training
s = (500, 500)
p1 = plotVector(loss_tr, s, :log10, "Loss function")
p2 = plotVector(acc_val, s, :none,  "f1-score")
display( plot(p1, p2, layout=(2,1), size=s) )

In [ ]:
# print metrics
ŷ = predictOutcome(X_val_g)
accuracyValidation, _ = printMetrics( ŷ, coerce(validationY, OrderedFactor) );

In [ ]:
# clean gpu memory
X_tr  = nothing
y_tr  = nothing
X_val = nothing
cleanCUDA()

Testing

In [ ]:
# preprocessing
X_tst, y_tst = preprocess1(testX, testY)
X_tst        = MLJ.transform(mach, X_tst) .|> Float32   # transform(unsupervised) vs predict(supervised)
X_tst, y_tst = preprocess2(X_tst, y_tst)
X_tst_g      = X_tst |> gpu

# predict
ŷ = predictOutcome(X_tst_g)

# result
accuracyTest, _ = printMetrics( ŷ, coerce(testY, OrderedFactor) );

In [ ]:
accuracyValidation > accuracyTest ? "fit ok" : "overfitting"

In [ ]:
# clean gpu memory
X_tst = nothing
cleanCUDA()